In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '137'
formatted_attribute = 'Weight Capacity'
customer_name='%kimballmidwest%'
buckets = """Office Chairs"""

strategy_version_input=767
attribute_id_input=2290

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
# fields = ['custom_fields']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
# df=dfs
print(len(df))
# reg_ex(df)

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')]

384


In [5]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [9]:
print(len(df[(df['Capacity'].astype(str)!='nan')&(df['Capacity'].astype(str)!='[]')]))

3


In [14]:
regex_pattern =r"(?i)(\d+.?lb)|()"
df['lb'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df_lb=df[df['lb'].astype(str)!='[]']

regex_pattern =r"(?i)(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}w?e?i?g?h?i?n?g?\s?\d+\s?lb)|((?<!weighs )(?<!\))(?<!\.)\d+(?:\.\d+)?(?:\,\d+)?\s?lb\.?\W{0,2}\s?p?o?u?n?d?\s?m?a?x?i?m?u?m?\s?s?t?a?t?i?c?\s?l?o?a?d?\s?d?i?s?t?r?i?b?u?t?e?d?\s?w?e?i?g?h?t?\s?s?e?a?t?i?n?g?\s?capacity(?! \d+ lb))|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb(?!\s?and.?up))|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(weight.?capacity.?o?f?.?.?\s?\d+\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?to\s?\d+.?\d*\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?-\s?\d+.?\d*\s?lb)|(up.?to.?\d+\.?\d*.?lb)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(store.?\s?up\s?to\s?\d+\s?lb)|(users\s?up\s?to\s\d+\s?lb)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|()"
df_lb['matches'] = df_lb['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [16]:
print(len(df_lb))

2


In [15]:
regex_lbs=r"(?i)(\d+\.?\d*\s?lb)|()"
df_lb['newest_matches']=df_lb['matches'].apply(lambda x: re_extract(regex_lbs, str(x)))

df_newest=df_lb[df_lb['newest_matches'].astype(str)!='[]']
df_newest['newest_matches']=df_newest['newest_matches'].apply(lambda x: natsorted(x))
df_newest['newest_matches']=df_newest['newest_matches'].apply(lambda x: re.sub(r"(?i) lb",'lb',str(x))).apply(lambda x: re.sub(r'\.lb','lb',str(x))).apply(lambda x: re.sub(r'\.lb','lb',str(x))).apply(lambda x: re.sub(r'lb',' lb',str(x))).apply(lambda x: re.sub(r'LB',' lb',str(x))).apply(lambda x: re.sub(r'  ',' ',str(x)))                 
df_newest['newest_matches']=df_newest['newest_matches'].apply(lambda x: re.sub(r'\[','["',str(x))).apply(lambda x: re.sub(r'\]','"]',str(x))).apply(lambda x: re.sub(r',','","',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r"'",'',str(x)))
curate(df_newest, 'newest_matches', formatted_attribute, 'ac-km-z')
df_newest[curation_col]=df_newest['newest_matches']
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'(?<!l)b','lb',str(x)))
rounds(df_newest,curation_col)
rounding(df_newest,curation_col,'ac-km-z')

Repeated External IDs: []


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, Type, Grade, Finish, Length, Diameter, Material, keywords, ChainSize, InsideLength, PrivateLabel, RatedCapacity, packageweight, packagequantity, shortdescription, Width, Pitch, IndustryNumber, Color, SafeWorkingLoad, ChokerRatingCapacity_LBS_, VerticalRatingCapacity_LBS_, VerticalBasketRatingCapacity_LBS_, Plies, WireDiameter, CoatedDiameter, _ofPieces, ThreadSize, ThroatWidth, ClevisPinSize, YokeWidth, Torque, CableSize, InsideDiameter, OutsideDiameter, EyeSize, ColorCode, TakeUp, Size, Capacity, lb, matches, newest_matches, Q:weight_capacity, rounding]
Index: []

In [13]:
df_newest

NameError: name 'df_newest' is not defined

In [10]:
matchesnewest=df_newest[['external_id',curation_col]]
print(len(df_newest))
# df_newest[0:5]

103


In [13]:
matchesnewest

NameError: name 'matchesnewest' is not defined

# Na

In [17]:
regex_pattern_na=r'(?i)(?i)(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}w?e?i?g?h?i?n?g?\s?\d+\s?lb)|((?<!weighs )(?<!\))(?<!\.)\d+(?:\.\d+)?(?:\,\d+)?\s?lb\.?\W{0,2}\s?p?o?u?n?d?\s?m?a?x?i?m?u?m?\s?s?t?a?t?i?c?\s?l?o?a?d?\s?d?i?s?t?r?i?b?u?t?e?d?\s?w?e?i?g?h?t?\s?s?e?a?t?i?n?g?\s?capacity(?! \d+ lb))|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb(?!\s?and.?up))|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(weight.?capacity.?o?f?.?.?\s?\d+\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?to\s?\d+.?\d*\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?-\s?\d+.?\d*\s?lb)|(up.?to.?\d+\.?\d*.?lb)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(store.?\s?up\s?to\s?\d+\s?lb)|(users\s?up\s?to\s\d+\s?lb)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)'   
df['matches_na'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

# regex_lb=r'(?i)(?<!weighs\s?)\d+.?\d*\s?lb'
# df['matches_lb'] = df['long_desc'].apply(lambda x: re_extract(regex_lb, x))

df_na=df[(df['matches_na'].astype(str)=='[]')]#&(df['matches_lb'].astype(str)!='[]')] 
df_na[curation_col]='n/a'
match_na=df_na[['external_id',curation_col]]

In [19]:
print(len(df_na))

384


In [21]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchesnewest))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_na))

match_na


In [23]:
print(len(df))
print(len(match_na))

384
384


In [14]:
rounding(df_newest, 'newest_matches','ac-km-z')

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, lb, matches, newest_matches, Q:weight_capacity, rounding]
Index: []

In [15]:
error

NameError: name 'error' is not defined

# send to the folder for upload

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('Kimball MidWest', attribute, df, match_na)